<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Model IO Exercise 

The purpose of this exercise is to test your understanding of building out Model IO systems. You will also hopefully notice the need to chain responses together, which we will cover later in this course!

Watch the video for a full overview on minimum outputs this class should be capable of, but feel free to expand on this project, or to just treat it as a code-along!

## History Quiz

Our main goal is to use LangChain and Python to create a very simple class with a few methods for:
* Writing a historical question that has a date as the correct answer
* Getting the correct answer from LLM
* Getting a Human user's best guess at at correct answer
* Checking/reporting the difference between the correct answer and the user answer

### Suggested Imports

Feel free to accomplish this task however you prefer!

In [123]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from datetime import datetime
from langchain_openai import OpenAI
from langchain.output_parsers import DatetimeOutputParser, OutputFixingParser
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os

In [12]:
load_dotenv()

True

In [16]:
api_key = os.environ['OPENAI_API_KEY']

In [20]:
model = ChatOpenAI(api_key=api_key)

In [46]:
output_parser = DatetimeOutputParser()
output_parser

DatetimeOutputParser()

In [209]:
class HistoryQuiz():
    
    def create_history_question(self,topic):
        '''
        This method should output a historical question about the topic that has a date as the correct answer.
        For example:
        
            "On what date did World War 2 end?"
            
        '''
        system_template = "You are a historical expert who specializes in {topic}"
        system_prompt = SystemMessagePromptTemplate.from_template(system_template)
        human_template = "Generate a single quiz question on {topic} that has a date as correct answer"
        human_prompt = HumanMessagePromptTemplate.from_template(human_template)
        chat_prompt_template = ChatPromptTemplate.from_messages([system_prompt,human_prompt])
        chat_prompt = chat_prompt_template.format_prompt(topic=topic).to_messages()
        result = model(chat_prompt)
        question = result.content
        return question
    
    def get_AI_answer(self,question):
        '''
        This method should get the answer to the historical question from the method above.
        Note: This answer must be in datetime format! Use DateTimeOutputParser to confirm!
        
        September 2, 1945 --> datetime.datetime(1945, 9, 2, 0, 0)
        '''
        system_template = "You are a history domain expert who needs to answer {question} with date as response"
        system_prompt = SystemMessagePromptTemplate.from_template(system_template)
        human_template = "{question}\n{format_instructions}"
        human_prompt = HumanMessagePromptTemplate.from_template(human_template)
        chat_prompt_template = ChatPromptTemplate(messages=[system_prompt,human_prompt],input_variables=['question','format_instructions'])
        chat_prompt = chat_prompt_template.format_prompt(question=question,format_instructions=output_parser.get_format_instructions()).to_messages()
        new_parser = OutputFixingParser.from_llm(llm=model,parser=output_parser)
        result = model(chat_prompt)
        correct_datetime = new_parser.parse(result.content)
        return correct_datetime
    
    def get_user_answer(self,question):
        '''
        This method should grab a user answer and convert it to datetime. It should collect a Year, Month, and Day.
        You can just use input() for this.
        '''
        print(question)
        year = int(input("Enter a year: "))
        month = int(input("Enter a month: "))
        day = int(input("Enter a day: "))
        user_datetime = datetime(year,month,day)
        return user_datetime
        
        
    def check_user_answer(self,user_answer,ai_answer):
        '''
        Should check the user answer against the AI answer and return the difference between them
        '''
        # print or return the difference between the answers here!
        print(f"The difference between the dates is: {user_answer-ai_answer}")
        

### Example Usage

Feel free to expand or edit this project. To keep things simple we have every method return an object that will then feed into a new method!

In [212]:
quiz_bot = HistoryQuiz()

In [214]:
question = quiz_bot.create_history_question(topic='FIFA World Cup')

In [216]:
question

'In which year did the FIFA World Cup tournament take place for the first time?'

In [218]:
ai_answer = quiz_bot.get_AI_answer(question)

In [220]:
# Day After Pearl Harbor
ai_answer

datetime.datetime(1930, 7, 13, 0, 0)

In [222]:
user_answer = quiz_bot.get_user_answer(question)

In which year did the FIFA World Cup tournament take place for the first time?


Enter a year:  1930
Enter a month:  5
Enter a day:  12


In [224]:
user_answer

datetime.datetime(1930, 5, 12, 0, 0)

In [226]:
quiz_bot.check_user_answer(user_answer,ai_answer)

The difference between the dates is: -62 days, 0:00:00
